### Text Generation using N-Grams model. 
The text corpus considered here is 'shakespeare.txt' which is available online and 
the same is divided into train and test set. The n-gram model has been trained on train set texts and predicted using test 
set texts. The n-gram model performance is evaluated using perplexity metric.


In [106]:
import os, string, re
import numpy as np
import math
import random
from random import shuffle
import nltk

In [107]:
with open("shakespeare.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("------")
display(data[0:300])
print("------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 4573338
First 300 letters of the data
------


"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us"

------
Last 300 letters of the data
-------


'his England never did, nor never shall,\nLie at the proud foot of a conqueror,\nBut when it first did help to wound itself.\nNow these her princes are come home again,\nCome the three corners of the world in arms,\nAnd we shall shock them. Nought shall make us rue,\nIf England to itself do rest but true.\n'

-------


In [108]:
# Split data into sentences.
def split_to_sentences(data):
    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]  # Remove leading and trailing spaces from each sentence
    sentences = [s for s in sentences if len(s) > 0]  # Drop sentences if they are empty strings.
    return sentences

In [109]:
# split a sentence into a list of words
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokenized = nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    return tokenized_sentences

In [110]:
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

In [111]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [114]:
print("{} Data are split into {} train and {} test set".format(len(tokenized_data), len(train_data), len(test_data)))
print("First training sample")
print(tokenized_train_data[0])

print("First test sample")
print(tokenized_test_data[0])

136177 Data are split into 108941 train and 27236 test set
First training sample
['first', 'citizen', ':']
First test sample
['i', 'did', 'never', 'see', 'it', '.']


In [120]:
def count_words(tokenized_sentences):
    word_counts = {}
    for sentence in tokenized_sentences:
        for token in sentence:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

In [121]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    # for each word and its count
    for word, cnt in word_counts.items():
        if cnt >= count_threshold:
            closed_vocab.append(word)
    return closed_vocab

In [122]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token = "<unk>"):
    vocabulary = set(vocabulary)
    # Initialize a list that will hold the sentences after less frequent words are replaced by the unknown token
    replaced_tokenized_sentences = []
    
    for sentence in tokenized_sentences:
        # Initialize the list that will contain a single sentence with "unknown_token" replacements
        replaced_sentence = []
        for token in sentence:
            if token in vocabulary:
                # If so, append the word to the replaced_sentence
                replaced_sentence.append(token)
            else:
                # otherwise, append the unknown token instead
                replaced_sentence.append(unknown_token)
        
        # Append the list of tokens to the list of lists
        replaced_tokenized_sentences.append(replaced_sentence)
        
    return replaced_tokenized_sentences

In [123]:
def preprocess_data(train_data, test_data, count_threshold):
    
    # Get the closed vocabulary using the train data
    vocabulary = get_words_with_nplus_frequency(train_data,count_threshold)
    
    # For the train data, replace less common words with "<unk>"
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    
    # For the test data, replace less common words with "<unk>"
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    
    return train_data_replaced, test_data_replaced, vocabulary

In [126]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, minimum_freq)

In [127]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['juliet', ':']

First preprocessed test sample:
['weapons', 'in', 'your', 'hand', ',', 'and', 'kill', 'me', 'a', '<unk>']

First 10 vocabulary:
['juliet', ':', 'don', 'adriano', 'de', 'armado', 'achilles', 'for', 'a', 'fish']

Size of vocabulary: 13236


In [128]:
def count_n_grams(data, n, start_token = '<s>', end_token = '<e>'):
    
    # Initialize dictionary of n-grams and their counts
    n_grams = {}
    
    for sentence in data:
        
        # prepend start token n times, and  append <e> one time
        sentence = [start_token] * n + sentence + [end_token]
        
        # convert list to tuple so that the sequence of words can be used as a key in the dictionary
        sentence = tuple(sentence)
        
        # Use 'i' to indicate the start of the n-gram from index 0 to the last index where the end of the n-gram
        # is within the sentence.
        for i in range(len(sentence) if n==1 else len(sentence)-1):
            
            # Get the n-gram from i to i+n
            n_gram = sentence[i:i+n]
            
            # check if the n-gram is in the dictionary
            if n_gram in n_grams.keys():
                
                # Increment the count for this n-gram
                n_grams[n_gram] += 1
            else:
                 # Initialize this n-gram count to 1
                n_grams[n_gram] = 1
    
    return n_grams

In [129]:
# Estimate the probabilities of a next word using the n-gram counts with k-smoothing
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    
    # convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)
    
    # Set the denominator. If the previous n-gram exists in the dictionary of n-gram counts, Get its count.  
    # Otherwise set the count to zero. Use the dictionary that has counts for n-grams.
    previous_n_gram_counts = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    
    # Calculate the denominator using the count of the previous n gram and apply k-smoothing
    denominator = previous_n_gram_counts + k * vocabulary_size
    
    # Define n plus 1 gram as the previous n-gram plus the current word as a tuple
    n_plus1_gram = previous_n_gram + (word,)
    
    # Set the count to the count in the dictionary,otherwise 0 if not in the dictionary
    # use the dictionary that has counts for the n-gram plus current word
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0
    
    # Define the numerator use the count of the n-gram plus current word,and apply smoothing
    numerator = n_plus1_gram_count + k
    
    # Calculate the probability as the numerator divided by denominator
    probability = numerator / denominator
    
    return probability
    
    

In [130]:
# function defined below loops over all words in vocabulary to calculate probabilities for all possible words.
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    
    # convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)
    
    # add <e> <unk> to the vocabulary. <s> is not needed since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability
        
    return probabilities

In [131]:
# The n-gram counts computed above are sufficient for computing the probabilities of the next word. 
# It can be more intuitive to present them as count or probability matrices.
# The functions defined in this cell return count matrices.
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    # add <e> <unk> to the vocabulary
    # <s> is omitted since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    
    # obtain unique n-grams
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    
    # mapping from n-gram to row
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}
    
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [ ]:
# The following function calculates the probabilities of each word given the previous n-gram, and stores this in matrix form.
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [132]:
# Generate the perplexity score to evaluate your model on the test set.
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    
    # length of previous words
    n = len(list(n_gram_counts.keys())[0])
    
    # prepend <s> and append <e>
    sentence = ["<s>"] * n + sentence + ["<e>"]
    
    # Cast the sentence from a list to a tuple
    sentence = tuple(sentence)
    
    # length of sentence (after adding <s> and <e> tokens)
    N = len(sentence)
    
    product_pi = 1.0
    
    for t in range(n, N):
        
        # get the n-gram preceding the word at position t
        n_gram = sentence[t-n:t]
        
        # get the word at position t
        word = sentence[t]
        
        probability = estimate_probability(word,n_gram, n_gram_counts, n_plus1_gram_counts, len(vocabulary_size), k=1)
        
        # Update the product of the probabilities. This 'product_pi' is a cumulative product 
        # of the (1/P) factors that are calculated in the loop
        product_pi *= 1 / probability
        
    # Take the Nth root of the product
    perplexity = product_pi**(1/float(N))
    
    return perplexity

In [133]:
# Compute probabilities for all possible next words and suggest the most likely one.
# This function also take an optional argument start_with, which specifies the first few letters of the next words.
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    
    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 
    
    # From the words that the user already typed get the most recent 'n' words as the previous n-gram
    previous_n_gram = previous_tokens[-n:]
    
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    
    # Initialize suggested word to None. This will be set to the word with highest probability
    suggestion = None
    
    # Initialize the highest word probability to 0. This will be set to the highest probability of all words to be suggested
    max_prob = 0
    
    # For each word and its probability in the probabilities dictionary:
    for word, prob in probabilities.items():
        
         # If the optional start_with string is set
        if start_with != None:
            
            # Check if the word starts with the letters in 'start_with'
            if not word.startswith(start_with):
                
                #If so, don't consider this word (move onto the next word)
                continue
                
        # Check if this word's probability is greater than the current maximum probability
        if prob > max_prob:
            
            # If so, save this word as the best suggestion (so far)
            suggestion = word
            
            # Save the new maximum probability
            max_prob = prob
            
    return suggestion, max_prob
            
    

In [134]:
# The function defined below loop over varioud n-gram models to get multiple suggestions.
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [135]:
# Let's see with n-grams of varying lengths (unigrams, bigrams, trigrams, 4-grams...6-grams).
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [136]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('the', 0.04270668013273698),
 ('stand', 0.00015087507543753772),
 ('stand', 0.00015088645794039985),
 ('<e>', 0.00015108022359873092)]

In [137]:
previous_tokens = ["i", "want", "to", "go"]
tmp_suggest5 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest5)

The previous words are ['i', 'want', 'to', 'go'], the suggestions are:


[(',', 0.015633571036752607),
 ('to', 0.0009778847600421243),
 ('juliet', 7.554011179936546e-05),
 ('juliet', 7.554011179936546e-05)]

In [147]:
previous_tokens = ["And", "we", "shall"]
tmp_suggest6 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with = 'shoc')

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest6)

The previous words are ['And', 'we', 'shall'], the suggestions are:


[('shock', 0.00012461059190031152),
 ('shock', 0.00014958863126402394),
 ('shocks', 7.554011179936546e-05),
 ('shocks', 7.554011179936546e-05)]

In [139]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest7 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest7)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[(',', 0.05488006617038875),
 ('?', 0.0017152658662092624),
 ('juliet', 7.554011179936546e-05),
 ('juliet', 7.554011179936546e-05)]

In [140]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('do', 0.007775020678246485),
 ('don', 7.457677679170706e-05),
 ('don', 7.554011179936546e-05),
 ('don', 7.554011179936546e-05)]